# Preprocess Data and Create Knowledge Graph

Insert neo4j db credentials here

In [4]:
from neo4j import GraphDatabase
URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "password")

### Prepare Ratings Dataframe

In [9]:
import pandas as pd

ratings = pd.read_csv("data/ml_small/ratings.csv")
ratings.drop(labels=["timestamp"], axis=1, inplace=True)
ratings.rename(columns={'userId': 'user_id', 'movieId': 'movie_id'}, inplace=True)
display(ratings)
nan_ratio = ratings.isna().mean()
print("NaN Ratios")
print(nan_ratio)

,user_id,movie_id,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


NaN Ratios
user_id     0.0
movie_id    0.0
rating      0.0
dtype: float64


### Sample
The original dataset includes > 100k ratings. This is too much to handle for my local machine. Therefore I apply a stratified split in order to not reduce the average number of ratings per user. You can change the variable `DIVISOR` according to the desired data size. 2 for example halves the original dataset, 3 cuts into thirds, etc.

In [2]:
from sklearn.model_selection import train_test_split
import numpy as np

DIVISOR = 2

# Step 1: Get unique user IDs
unique_user_ids = ratings['user_id'].unique()

# Step 2: Shuffle the user IDs
np.random.shuffle(unique_user_ids)

# Step 3: Split the user IDs into two parts
split_index = len(unique_user_ids) // DIVISOR
user_ids_split_1 = unique_user_ids[:split_index]
user_ids_split_2 = unique_user_ids[split_index:]

# Step 4: Create the splits in the DataFrame
df_split_1 = ratings[ratings['user_id'].isin(user_ids_split_1)]
df_split_2 = ratings[ratings['user_id'].isin(user_ids_split_2)]

# Step 5: Train test split stratified on user_id column so that all user_ids of test set are known already in train set
ratings_reduced = df_split_1
ratings_train, ratings_test = train_test_split(ratings_reduced, test_size=0.3, stratify=ratings_reduced['user_id'], random_state=11717659)

assert set(ratings_train['user_id']) == set(ratings_test['user_id'])

unique_movie_ids = ratings_reduced["movie_id"].unique()
print(len(ratings_reduced["movie_id"].unique()))
print(len(ratings_reduced["user_id"].unique()))
print(len(ratings_reduced.index))

6722
305
45211


### Prepare Movies Dataframe
Remove movies that are not rated

In [3]:
movies = pd.read_csv("data/ml_small/movies.csv")
movies.rename(columns={'movieId': 'movie_id'}, inplace=True)
# Remove movies that are are never rated in train or test set
movies = movies[movies["movie_id"].isin(unique_movie_ids)]

movies["genres"] = movies["genres"].str.split("|")
movies["year"] = movies["title"].str.extract(r'\((\d{4})\)')
movies['year'] = pd.to_numeric(movies['year'], errors='coerce')
movies['year'] = movies['year'].fillna(movies["year"].median()).astype(int)
movies["genres"] = movies["genres"].apply(lambda x: [] if x == ["(no genres listed)"] else x)

display(movies)
nan_ratio = movies.isna().mean()
print("NaN Ratios")
print(nan_ratio)

,movie_id,title,genres,year
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II (1995),[Comedy],1995
...,...,...,...,...
9736,193579,Jon Stewart Has Left the Building (2015),[Documentary],2015
9737,193581,Black Butler: Book of the Atlantic (2017),"[Action, Animation, Comedy, Fantasy]",2017
9738,193583,No Game No Life: Zero (2017),"[Animation, Comedy, Fantasy]",2017
9739,193585,Flint (2017),[Drama],2017


NaN Ratios
movie_id    0.0
title       0.0
genres      0.0
year        0.0
dtype: float64


### Populate Neo4j Database as Knowledge Graph

In [5]:
def reset_db(driver):
    driver.execute_query(
    "MATCH (n)"
    "DETACH DELETE n",
    database_="neo4j")

In [6]:
def create_movies(driver):
    counter = 0
    movies_len = movies.shape[0]
    for _, row in movies.iterrows():
        print(f"Relations added: {counter} / {movies_len}", end="\r")
        counter += 1
        driver.execute_query(
            """
            MERGE (m:Movie {movie_id: $movie_id, title: $title})
            MERGE (y:Year {year: $year})
            MERGE (m)-[:RELEASED_IN]->(y)
            """,
            movie_id = row["movie_id"], title = row["title"],
            year = row["year"], database_ = "neo4j")
        for genre in row["genres"]:
            driver.execute_query(
                """
                MATCH (m:Movie {movie_id: $movie_id})
                MERGE (g:Genre {name: $genre})
                MERGE (m)-[:OF_GENRE]->(g)
                """,
                movie_id = row["movie_id"], title = row["title"],
                genre = genre, year = row["year"], database_ = "neo4j")
    print("Movie and year nodes and RELEASED_IN relation added.")

In [7]:
def create_user_ratings(driver):
    counter = 0
    ratings_train_len = ratings_train.shape[0]
    for _, row in ratings_train.iterrows():
        if row["rating"] == 3:
            continue
        print(f"Relations added: {counter} / {ratings_train_len}", end="\r")
        counter += 1
        driver.execute_query(
            f"""
            MERGE (u:User {{user_id: $user_id}})
            WITH u
            MATCH (m:Movie {{movie_id: $movie_id}})
            MERGE (u)-[:RATED {{rating: $rating}}]->(m)
            """,
            user_id = row["user_id"], movie_id = row["movie_id"],
            rating = row["rating"], database_ = "neo4j")
    print("Rating relations and user nodes added.")

In [8]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    reset_db(driver)
    create_movies(driver)
    create_user_ratings(driver)

Movie and year nodes and RELEASED_IN relation added.
Rating relations and user nodes added.
